# Mesogens with NP | Pressure Ramps

## Pressure: 1.6 -> 1.8 

### Date: 11/13/2019 | System P = 1.8, Expected value of $T_c$ : 7.09 

In [1]:
#-----Importa los paquetes esenciales para correr la simulación.

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables

p_prev = 1.6;
p_new = 1.8;
temp = 6.0;
steps_ramp = 5e4;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(temp) + "_P_" + str(p_prev) + "_ramp.gsd"

In [4]:
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd(ramp_file,frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#-----Integrate using NPT

pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (steps_ramp, p_new)])
npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 2.0, tauP = 2.0, P = pressure);
npt.randomize_velocities(seed = 42)

#-----Define los archivos en los que se guardará la información de la simulación. 

log_file = "T_" + str(temp) + "_P_" + str(p_new) + "_ramp.log"
gsd_file = "T_" + str(temp) + "_P_" + str(p_new) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(temp) + "_P_" + str(p_new) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [5]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 394170 / 444000 | TPS 16.9631 | ETA 00:48:57
Time 00:00:20 | Step 394372 / 444000 | TPS 20.1906 | ETA 00:40:57
Time 00:00:30 | Step 394559 / 444000 | TPS 18.6214 | ETA 00:44:15
Time 00:00:40 | Step 394749 / 444000 | TPS 18.9851 | ETA 00:43:14
Time 00:00:50 | Step 394945 / 444000 | TPS 19.4295 | ETA 00:42:04
Time 00:01:00 | Step 395149 / 444000 | TPS 20.2371 | ETA 00:40:13
Time 00:01:10 | Step 395343 / 444000 | TPS 19.3991 | ETA 00:41:48
Time 00:01:20 | Step 395544 / 444000 | TPS 19.9214 | ETA 00:40:32
Time 00:01:30 | Step 395735 / 444000 | TPS 19.0787 | ETA 00:42:09
Time 00:01:40 | Step 395929 / 444000 | TPS 19.2908 | ETA 00:41:31
Time 00:01:50 | Step 396123 / 444000 | TPS 19.3201 | ETA 00:41:18
Time 00:02:00 | Step 396324 / 4

In [6]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x11630a978>),
             ('particles', <hoomd.data.particle_data at 0x11638bd68>),
             ('number_density', 0.6264435925493604),
             ('bonds', <hoomd.data.bond_data at 0x11638bdd8>),
             ('angles', <hoomd.data.angle_data at 0x11638be48>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x11638beb8>),
             ('impropers', <hoomd.data.dihedral_data at 0x11638bf28>),
             ('constraints', <hoomd.data.constraint_data at 0x11638bf98>),
             ('pairs', <hoomd.data.bond_data at 0x116435048>),
             ('timestep', 444000)])

In [7]:
#-----Get volume.

system.box.get_volume()

14373.201525387989